In [1]:
import numpy as np
from scipy.signal import convolve
import math as ma
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
global Nr, Nc, grid, boundarymask, exitmask, snakemask, snakepos, laplacian, dirs

Nr = 550
Nc = ma.ceil(Nr*8.5/11)

grid = np.zeros((Nr,Nc), dtype=np.float64)

boundarymask = np.array([[0 if r == 0 or r == Nr-1 or c == 0 or c == Nc-1 else 1 for c in range(Nc)] for r in range(Nr)], dtype=np.float64)
exitmask = np.array([[1 if r == Nr//2 and c == Nc-1 else 0 for c in range(Nc)] for r in range(Nr)], dtype=np.float64)
snakemask = np.array([[0 if r == Nr//2 and c <= 1 else 1 for c in range(Nc)] for r in range(Nr)], dtype=np.float64)
snakepos = [np.array([Nr//2,0]),np.array([Nr//2,1])]

laplacian = 0.25*np.array([[0,1,0],[1,0,1],[0,1,0]], dtype=np.float64)

dirs = np.array([[1,0],[0,1],[-1,0],[0,-1]])

In [3]:
def averaging_step(grid):
    grid = convolve(grid,laplacian,mode='same')
    grid = np.maximum(np.minimum(np.minimum(grid,boundarymask), snakemask),exitmask)
    return grid

def calculate_harmonic_function(grid, num_iterations=Nr*Nc):
    for _ in range(num_iterations):
        grid = averaging_step(grid)
    return grid

def walk_snake(grid):
    weights = np.array([grid[*(snakepos[-1] + x)] for x in dirs])
    weights = weights / np.sum(weights)
    dir = dirs[np.random.choice(len(dirs), p=weights)]
    snakepos.append(snakepos[-1] + dir)
    snakemask[*snakepos[-1]] = 0
    grid = np.maximum(np.minimum(np.minimum(grid,boundarymask),snakemask),exitmask)
    return grid, snakepos

In [ ]:
for i in tqdm(range(1001)):
    if (snakepos[-1] != np.array([Nr//2,Nc-1])).any():
        grid = calculate_harmonic_function(grid)
        grid, snakepos = walk_snake(grid)
    
    if i % 10 == 0:
        plt.close()
        plt.matshow(grid, cmap='gist_gray')
        plt.matshow(snakemask, cmap='binary')